## Add Custom Narrative to FHIR Search Parameter

- Get SP from file
- Generate Narrative from Jinja2 Template
- Strip out any null values
- Save SP back to file

In [5]:
from json import dumps, loads, load
from pathlib import Path
from IPython.display import display as Display, HTML, Markdown
from jinja2 import Environment, FileSystemLoader, select_autoescape
from lxml import etree
from fhir.resources import construct_fhir_element
from commonmark import commonmark
from datetime import datetime

def json_serial(obj):
    """JSON serializer for datetime objects not serializable by default json code"""

    if isinstance(obj, (datetime)):
        return obj.strftime('%Y-%m-%d')
    raise TypeError ("Type %s not serializable" % type(obj))

### Get SP from file

In [6]:
in_path = Path() / 'test' / 'in_sp.json' # temp path
sp = loads(
        in_path.read_text()
    )
sp['date'] = datetime.utcnow()  #convert to dateime object
sp

{'resourceType': 'SearchParameter',
 'id': 'us-core-race',
 'text': {'div': '<div xmlns="http://www.w3.org/1999/xhtml"><!--<h2>SearchParameter: USCoreRace</h2>--><b> description</b> : <p>Returns patients with a race extension matching the specified code.</p>\n\n\t\t\t<br></br>\n\t\t\t<p><b> id</b> us-core-race</p>\n\t\t\t<p><b> url</b> : <b> http://hl7.org/fhir/us/core/SearchParameter/us-core-race</b>\n\t\t\t</p>\n\t\t\t<p><b> version</b> : 4.1.0</p>\n\t\t\t<p><b> name</b> : USCoreRace</p>\n\t\t\t\n\t\t\t<p><b> status</b> : active</p>\n\t\t\t\n\t\t\t<p><b> date</b> : 2019-05-21</p>\n\t\t\t<p><b> publisher</b> : HL7 International - US Realm Steering Committee</p>\n\t\t\t<p><b> contact</b> : http://www.hl7.org/Special/committees/usrealm/index.cfm</p>\n            <!--\n\t\t\t<p>\n\t\t\t\t<b> useContext</b> : </p>\n            -->\n\t\t\t<p><b> jurisdiction</b> : United States of America (the) <span> (Details : {urn:iso:std:iso:3166 code \'US\' = \'United States of America\', given as \'U

### Generate Narrative from Jinja2 Template

In [7]:
def markdown(text, *args, **kwargs):
    return commonmark(text, *args, **kwargs)

env = Environment(
    loader=FileSystemLoader(searchpath = ''),
    autoescape=select_autoescape(['html','xml','xhtml','j2','md'])
    )


env.filters['markdown'] = markdown

template = env.get_template('sp_narrative.j2')  

rendered = template.render(sp=sp)
display(HTML(rendered))


parser = etree.XMLParser(remove_blank_text=True)
root = etree.fromstring(rendered, parser=parser)

div = (etree.tostring(root[1][0], encoding='unicode', method='html'))

my_obj = dict(
            status = 'generated',
            div = div
            )
            
narr = construct_fhir_element('Narrative', my_obj)

sp['text'] = narr.dict()

### Save SP back to file

In [8]:
out_path = Path() / 'test' / 'out_sp.json' # temp path
out_path.write_text(dumps(sp, indent = 2, default=json_serial))

3336